In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn

from scipy.stats import shapiro

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, RobustScaler, StandardScaler
from sklearn.feature_selection import f_classif, chi2, SelectKBest
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_recall_curve, roc_auc_score, precision_score, recall_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
DATA_DIR = '/kaggle/input/sf-scoring/'
train_df = pd.read_csv(DATA_DIR + '/train.csv')
test_df = pd.read_csv(DATA_DIR + '/test.csv')
submission = pd.read_csv(DATA_DIR + '/sample_submission.csv')

## Описание полей

`client_id` - идентификатор клиента

`education` - уровень образования

`sex` - пол заемщика

`age` - возраст заемщика

`car` - флаг наличия автомобиля

`car_type` - флаг автомобиля иномарки

`decline_app_cnt` - количество отказанных прошлых заявок

`good_work` - флаг наличия “хорошей” работы

`bki_request_cnt` - количество запросов в БКИ

`home_address` - категоризатор домашнего адреса

`work_address` - категоризатор рабочего адреса

`income` - доход заемщика

`foreign_passport` - наличие загранпаспорта

`sna` - связь заемщика с клиентами банка

`first_time` - давность наличия информации о заемщике

`score_bki` - скоринговый балл по данным из БКИ

`region_rating` - рейтинг региона

`app_date` - дата подачи заявки

`default` - флаг дефолта по кредиту

In [ ]:
display(
    train_df.info(),
    train_df.describe(),
    train_df.describe(include=['object']),
    train_df.isna().sum().sort_values(ascending=False),
    train_df.head(),
)

Пойдём по признакам по порядку и исследуем их.

Предварительно сформируем несколько пустых списков с признаками каждого типа (категориальные, числовые, бинарные).

In [ ]:
std_cols = []
robust_cols = []
ordinal_cols = []
one_hot_cols = []
bin_cols = []
cols_to_delete = []
log_cols = []
na_fill_values = {}

In [ ]:
# !pip install pandas-profiling

In [ ]:
# from pandas_profiling import ProfileReport

# ProfileReport(train_df)

### `client_id`

Идентификатор клиента.

Неинформативный признак.

*Будем удалять.*

In [ ]:
cols_to_delete.append('client_id')

### `age`

Возраст клиента. Числовой признак.

Визуализируем его в разрезе дефолта/не-дефолта.

In [ ]:
def get_axes(figsize=None):
    fig = plt.figure(figsize=figsize or (10, 5))
    
    return fig.add_axes([0, 0, 1, 1])

def build_histplot(x, hue=None, ax=None, palette=None, bins='auto'):
    return seaborn.histplot(
        x=x,
        hue=hue,
        bins=bins,
        palette=palette,
        multiple='stack',
        ax=ax or get_axes(),
    )

def build_boxplot(x, y, ax=None):
    return seaborn.boxplot(x=x, y=y, orient='h', ax=ax or get_axes())

def build_distplots(x, y, shape=(2, 1), figsize=None, palette=None):
    fig = plt.figure(figsize=figsize or (6 * shape[0], 7 * shape[1]))
    axes = fig.subplots(*shape)

    boxplot = build_boxplot(x, y, ax=axes[0])
    histplot = build_histplot(x, y, ax=axes[1], palette=palette)

    return axes

build_distplots(
    train_df['age'],
    train_df['default']
);

In [ ]:
def build_barplot(x, y, scale=(0, 1), step=0.1, figsize=None):
    plot = seaborn.barplot(y=y, x=x, ax=get_axes(figsize))
    plot.set_yticks(np.arange(scale[0], scale[1] + step, step))
    plot.grid()

    return plot

default_by_age = train_df.groupby('age', as_index=False)['default'].mean()
    
build_barplot(default_by_age['age'], default_by_age['default']);

Из построенных графиков можем видеть, что медианный возраст дефолтных клиентов ниже. Явных выбросов по распределению не наблюдается. В процентном соотношении количество дефолтных клиентов в каждой возрастной группе варьируется в районе 11-14%. В старшей возрастной группе есть подозрительные значения (8-16-50%, возможно вызванные невысоким количеством наблюдений в этих группах). Взглянем на них поближе.

In [ ]:
for i in [68, 69, 70, 71, 72]:
    display(train_df[train_df['age'] == i])

Из выведенных графиков и таблиц можем увидеть, что наблюдений для клиентов в возрасте больше 70 очень мало, и поэтому количество дефолтных клиентов в них сильно отклоняется от количества клиентов в более младших возрастных группах. Это может исказить результат моделирования, поэтому удалим эти наблюдения из тренировочной выборки.

Проведём тест на нормальность распределения.

In [ ]:
def check_normal_distribution(values, alpha=0.05):
    H0 = 'Данные распределены нормально'
    Ha = 'Данные распределены не нормально'

    p_value = shapiro(values)[0]
    is_normal_distribution = p_value > alpha
    
    if is_normal_distribution:
        print(f'p-value: {p_value} > {alpha}. {H0}')
    else:
        print(f'p-value: {p_value} < {alpha}. {Ha}')
        
    return is_normal_distribution

check_normal_distribution(train_df['age']);

В соответствии с проведённым тестом, распределение данных близко к нормальному. Произведём поиск выбросов методом z-отклонений.

In [ ]:
def outliers_z_score(values):
    mean = np.mean(values)
    std = np.std(values)
    delta = 3 * std
    lower_bound = mean - delta
    upper_bound = mean + delta

    return values[(values < lower_bound) | (values > upper_bound)]

train_df = train_df.drop(outliers_z_score(train_df['age']).index)

Впоследствии применим к этому признаку `StandardScaler`.

In [ ]:
std_cols.append('age')

### `education`

Уровень образования клиента. Категориальный признак.

По возрастанию уровня образования:

`SCH` - среднее образование

`UGR` - аналог бакалавра

`GRD` - магистр

`PGR` - магистр+стажировка

`ACD` - высшая категория образования

Единственный признак с пропусками в датасете.

Устраним пропуски, проверим его на наличие выбросов, визуализируем в разрезе дефолта/не-дефолта и произведём его кодировку.

Посмотрим общее соотношение уровня образования в датасете.

In [ ]:
train_df['education'].value_counts(normalize=True) * 100

Заполнять пропуски будем модой (`SCH`)

In [ ]:
na_fill_values['education'] = train_df['education'].mode()[0]
train_df['education'] = train_df['education'].fillna(na_fill_values['education'])

In [ ]:
build_histplot(train_df['education'], train_df['default']);

In [ ]:
default_by_education = train_df.groupby('education', as_index=False)['default'].mean()

display(default_by_education)
build_barplot(default_by_education['education'], default_by_education['default'], step=0.05);

Из графиков можем видеть, что большая часть клиентов имеет среднее образование (`SCH`) и очень малая часть имеет образование уровня `ACD` и `PGR` (0.26% и 1.7% соответственно), что не является чем-то необычным. Также у этих уровней образования наблюдается самый низкий процент дефолтов (в 2-3 раза ниже, чем у `SCH` и `UGR`). Несмотря на малое количество наблюдений для этих уровней образования, считать их выбросами не будем, т.к. это может являться весомым фактором для определения платёжеспособности клиента.

*К этому признаку будем применять `OneHotEncoder`.*

*Потенциальный вариант улучшения моделирования: заполнять пропуски не модой, а на основе предсказний с помощью машинного обучения.*

In [ ]:
one_hot_cols.append('education')

### `sex`

Пол клиента. Бинарный признак.

Визуализируем в разрезе дефолт/не-дефолт.

In [ ]:
display(train_df['sex'].value_counts(normalize=True))
build_histplot(train_df['sex'], train_df['default']);

In [ ]:
default_by_sex = train_df.groupby('sex', as_index=False)['default'].mean()

display(default_by_sex)
build_barplot(default_by_sex['sex'], default_by_sex['default']);

Соотношение женщин к мужчинам в датасете 43 к 56. Для каждого пола процент дефолтных клиентов 12-13%. Ничего необычного в содержимом признака нет.

*Будем перекодировать его в цифровой формат.*

In [ ]:
bin_cols.append('sex')

### `car`

Флаг наличия автомобиля у клиента. Бинарный признак. 

Визуализируем признак в разрезе дефолт/не-дефолт.

In [ ]:
display(train_df['car'].value_counts(normalize=True))
build_histplot(train_df['car'], train_df['default']);

In [ ]:
default_by_car = train_df.groupby('car', as_index=False)['default'].mean()

display(default_by_car)
build_barplot(default_by_car['car'], default_by_car['default']);

Соотношение клиентов с машиной к клиентам без машины 1 к 3. В разрезе дефолта/не-дефолта соотношение дефолтных к не-дефолтным клиентам на 3% больше среди клиентов без машины.

### `car_type`

Флаг наличия автомобиля-иномарки. Бинарный признак.

Вероятно, имеет тесную связь с признаком `car`.

Визуализируем признак в разрезе дефолт/не-дефолт, проверим связь с `car_type`, проверим на наличие выбросов.

In [ ]:
display(train_df['car_type'].value_counts(normalize=True))
build_histplot(train_df['car_type'], train_df['default']);

In [ ]:
default_by_car_type = train_df.groupby('car_type', as_index=False)['default'].mean()

display(default_by_car_type)
build_barplot(default_by_car_type['car_type'], default_by_car_type['default']);

Также посмотрим на этот признак в разрезе клиентов с машинами.

In [ ]:
clients_with_cars = train_df[train_df['car'] == 'Y']
clients_with_cars['car_type'].value_counts(normalize=True)

In [ ]:
default_by_clients_with_cars_car_type = clients_with_cars.groupby('car_type', as_index=False)['default'].mean()

display(default_by_clients_with_cars_car_type)
build_barplot(
    default_by_clients_with_cars_car_type['car_type'],
    default_by_clients_with_cars_car_type['default'],
);

Соотношение клиентов без иномарок к клиентам с иномарками 4 к 1. В разрезе клиентов с машинами соотношение клиентов с иномарками и клиентам без иномарок 3 к 2 соответственно. В разрезе дефолта/не-дефолта соотношение дефолтных клиентов к не-дефолтным на 5% больше среди клиентов без иномарок вне зависимости от наличия автомобиля.

`car_type` целиком завязан на признак `car` и не может быть положительным без положительного `car`. На этом основании проверим данные признаки на выбросы.

In [ ]:
train_df[(train_df['car'] == 'N') & (train_df['car_type'] == 'Y')].shape[0]

Выбросов среди признаков `car` и `car_type` нет.

*Для удобства в будущем переделаем признаки `car` и `car_type` в признаки `has_domestic_car` и `has_foreign_car`. Потенциально у клиентов могут быть 2 и больше машин - иномарки и отечественные, но, к сожалению, из предоставленных данных нет возможности узнать это, поэтому просто поделим клиентов на клиентов без машины, с отечественным авто и с иномаркой.*

In [ ]:
cols_to_delete.extend(['car', 'car_type'])

### `decline_app_cnt`

Количество отклонённых заявок в прошлом. Числовой признак.

Визуализируем в разрезе дефолт/не-дефолт, проверим на выбросы.

In [ ]:
build_distplots(train_df['decline_app_cnt'], train_df['default']);

Большинство наблюдений нулевые или находятся около нуля. Наблюдается некоторое количество наблюдений с большими отклонениями для обоих типов клиентов. Среди дефолтных клиентов 75-й квартиль распределения больше, чем у не-дефолтных, т.е. дефолтным клиентам чаще отказывают.

Ознакомимся с наблюдениями с сильными отклонениями для данного показателя (начнём с 20 и будем понижать порог) и сравним их с данными из всего датасета, а также оценим корреляцию данного признака с целевым.

In [ ]:
display(train_df.describe(), train_df[['decline_app_cnt', 'default']].corr())

for i in np.arange(20, 4, -5):
    filtered_data = train_df[train_df['decline_app_cnt'] > i]

    display(f'decline_app_cnt > {i}', filtered_data, filtered_data.describe(), filtered_data[['decline_app_cnt', 'default']].corr())

Из построенных таблиц видим, что клиентов с большим количеством отклонённых заявок (больше 5) - несколько сотен, при этом корреляция данного признака с целевым из подвыборок ниже, чем в основном датасете и находится около нуля, т.е. прямой взаимосвязи между признаками нет.

Проведём тест на нормальность распределения.

In [ ]:
check_normal_distribution(train_df['decline_app_cnt'])

Результат теста неоднозначный - распределение данных близко к нормальному, однако на графиках мы этого не наблюдаем.

Пока будем считать, что данные распределены ненормально, поэтому поиск выбросов через z-отклоенение отбрасываем. Т.к. 25 и 75 квартиль равны нулю, мы также не можем воспользоваться методом Тьюки. Поэтому установим порог в 5 отказов и уберём наблюдения, где значение превышает этот порог.

In [ ]:
train_df = train_df.drop(train_df[train_df['decline_app_cnt'] > 5].index)

*В дальнейшем применим к нему `RobustScaler`, т.к. распределение ненормальное.*

*Потенциальный вариант улучшения модели: переделать признак в категориальный (0, 1-2, 3-4, 5+), либо установить вместо всех высоких значений значение 5. Также можно логарифмировать признак и удалять выбросы с методом z-отклонений.*

In [ ]:
robust_cols.append('decline_app_cnt')

### `good_work`

Флаг наличия "хорошей" работы (что бы это ни значило). Бинарный признак.

Визуализируем его в разрезе дефолт/не-дефолт.

In [ ]:
display(train_df['good_work'].value_counts(normalize=True) * 100)
build_histplot(train_df['good_work'], train_df['default'], bins=train_df['good_work'].nunique());

In [ ]:
default_by_good_work = train_df.groupby('good_work', as_index=False)['default'].mean()

display(default_by_good_work)
build_barplot(default_by_good_work['good_work'], default_by_good_work['default']);

"Хорошая" работа есть только у 16% клиентов, представленных в датасете. Среди клиентов с "хорошей" работой дефолтов меньше на 3%.

*Признак в кодировке не нуждается.*

In [ ]:
bin_cols.append('good_work')

### `bki_request_cnt`

Количество запросов в БКИ. Числовой признак.

Визуализируем в разрезе дефолт/не-дефолт, проверим на наличие выбросов. Также проверим связь с `decline_app_cnt`.

In [ ]:
display(train_df['bki_request_cnt'].value_counts(normalize=True) * 100)
build_distplots(train_df['bki_request_cnt'], train_df['default']);

На графиках видим, что в признаке есть достаточно большие значения для обоих категорий клиентов, которые, вероятно, могут являться выбросами. Ознакомимся с ними. Начнём с установления порога в 50 заявок и будем снижать до 5 с шагом в 5.

In [ ]:
display(train_df.describe(), train_df[['bki_request_cnt', 'default']].corr())

for i in np.arange(50, 4, -5):
    filtered_data = train_df[train_df['bki_request_cnt'] > i]

    print(f'bki_request_cnt > {i}'.center(50, '-'))

    display(filtered_data[['bki_request_cnt', 'default']].corr(), filtered_data.describe(), filtered_data)

Из выведенных таблиц можем видеть, что количество наблюдений с значением признака >10 относительно маленькое - чуть больше полусотни. Корреляция с целевым признаком практически отсутствует. Настолько высокие значения могут вносить искажения в работу модели. Но в подвыборках наблюдается более высокий процент дефолтных клиентов, что может сигнализировать, что помимо количества запросов в БКИ у них есть ещё какие-то признаки, позволяющие оценить их платёжеспособность.

Проведём тест на нормальность распределения.

In [ ]:
check_normal_distribution(train_df['bki_request_cnt'])

Как и с признаком `decline_app_cnt`, результаты теста на нормальность распределения неоднозначны и не стыкуются с графиком. Пока будем считать, что данные распределены ненормально.

Удалим выбросы с помощью метода Тьюки.

In [ ]:
def outliers_iqr(values):
    quantile_1, quantile_3 = np.quantile(values, 0.25), np.quantile(values, 0.75)
    iqr = quantile_3 - quantile_1
    
    return values[(values < quantile_1 - (iqr * 1.5)) | (values > quantile_3 + (iqr * 1.5))]

train_df = train_df.drop(outliers_iqr(train_df['bki_request_cnt']).index)

*В дальнейшем применим к признаку `RobustScaler`.*

*Потенциальные варианты усовершенствования модели: перевести этот признак в категориальный (0 запросов, 1-3, 3-6, 7+) и обойтись без удаления выбросов, либо подставить вместо высоких значений значение 7. Также можно попробовать логарифмировать признак и удалять выбросы через z-отклонение, т.к. тест на нормальность распределения неоднозначный.*

In [ ]:
robust_cols.append('bki_request_cnt')

### `home_address`

Категоризатор домашнего адреса. Категориальный признак.

Исследуем его, посмотрим соотношение, визуализируем в разрезе дефолт/не-дефолт.

In [ ]:
display(train_df['home_address'].value_counts(normalize=True) * 100)
build_histplot(train_df['home_address'], train_df['default'], bins=train_df['home_address'].nunique());

In [ ]:
default_by_home_address = train_df.groupby('home_address', as_index=False)['default'].mean()

display(default_by_home_address)
build_barplot(default_by_home_address['home_address'], default_by_home_address['default']);

`home_address` с значением 3 является самым редким в количестве 1.7%, значения 1 и 2 находятся в соотношении 44 к 54 соответственно. Однако процент дефолтов не сильно варьируется от типа к типу. Самый высокий процент у значения 2, после него идёт 3, на последнем месте 1 с соотношениями 15%, 11% и 9% соответственно.

Выбросы тут искать бесполезно, т.к. нет расшифровки значений адреса.

*В будущем закодируем этот признак с помощью `OneHotEncoding`, т.к. вряд ли адрес 1 типа качественно хуже адреса с типом 2.*

In [ ]:
one_hot_cols.append('home_address')

### `work_address`

Категоризатор рабочего адреса. Категориальный признак.

Исследуем его, посмотрим соотношение, визуализируем в разрезе дефолт/не-дефолт.

In [ ]:
display(train_df['work_address'].value_counts(normalize=True) * 100)
build_histplot(train_df['work_address'], train_df['default'], bins=train_df['work_address'].nunique());

In [ ]:
default_by_work_address = train_df.groupby('work_address', as_index=False)['default'].mean()

display(default_by_work_address)
build_barplot(default_by_work_address['work_address'], default_by_work_address['default']);

Из построенных графиков видим, что соотношение типов рабочих адресов близко к 6/3/1 для адресов 3, 2 и 1 типа соответственно. В разрезе дефолт/не-дефолт видится некая зависимость платёжеспособности клиента от типа рабочего адреса - самый малочисленный 1 тип адреса имеет в 2 раза меньший процент дефолтов  по сравнению с 3 типом, который представлен в наибольшем количестве (7% против 14%). 2 тип находится посередине и имеет 10% дефолтов.

Проверим связь этого признака с признаком `good_work`.

In [ ]:
display(train_df.groupby('work_address', as_index=False)['good_work'].mean())
build_histplot(train_df['work_address'], train_df['good_work'], bins=train_df['work_address'].nunique());

"Хорошая" работа чаще встречается у клиентов с 1-м, самым малочисленным, типом рабочего адреса (19%). Но не сказать что это количество значительно выше, чем у двух других типов (15% и 17%)

In [ ]:
train_df[['good_work', 'work_address']].corr(method='pearson')

Корреляции между признаками нет.

Выбросы искать тут бесполезно.

*В будущем закодируем признак с помощью `OneHotEncoder`, т.к. неизвестно, является ли тип рабочего адреса 1 качественно хуже типа 2.*

In [ ]:
one_hot_cols.append('work_address')

### `income`

Доход заёмщика. Числовой признак.

Исследуем его, визуализируем в разрезе дефолт/не-дефолт, поищем выбросы.

In [ ]:
build_distplots(train_df['income'], train_df['default']);

График распределений сконцентрирован около нуля, т.к. есть очень высокие значения (до 1 миллиона). Попробуем сначала поискать выбросы, сопоставляя доход с другими признаками (`good_work`, `work_address`, `age`), чтобы хоть как-то снизить искажения средних значений, после этого попробуем логарифмировать признак.

In [ ]:
build_boxplot(train_df['income'], train_df['good_work']);

In [ ]:
build_boxplot(train_df['income'], train_df['work_address']);

In [ ]:
build_boxplot(train_df['income'], train_df['age'], ax=get_axes(figsize=(10, 15))).grid();

Дополнительно построим матрицу корреляций всех этих признаков.

In [ ]:
train_df[['income', 'age', 'work_address', 'good_work']].corr(method='pearson')

По построенным графикам можно увидеть, что клиенты с "хорошей" работой имеют немного больший медианный заработок, однако количество высоких значений признака больше у клиентов без "хорошей" работы (либо они честнее). Также среди типов рабочих адресов больший медианный заработок имеют клиенты с 3 типом рабочего адреса. В плане высоких значений дохода не прослеживается зависимости от типа рабочего адреса - для всех 3 типов разброс значений примерно одинаковый. Медианный заработок в зависимости от возраста примерно одинаков для всех возрастов, кроме возрастов младше 23 и старше 54, для которых этот показатель немного снижается.

К выбросам потенциально можно отнести клиентов в возрасте 21-24 лет с доходом 200+ тыс. и клиентов с доходом 500+ тыс., т.к. таких людей относительно немного. Оставим эту опцию на будущее как возможность улучшения модели.

Логарифмируем признак и визуализируем его.

In [ ]:
income_log = np.log(train_df['income'])

build_distplots(income_log, train_df['default']);

Проверим распределение логарифмированных данных на нормальность.

In [ ]:
check_normal_distribution(income_log);

Будем считать, что данные распределены нормально. Воспользуемся методом z-отклонений для удаления выбросов на логарифмированном признаке.

In [ ]:
train_df = train_df.drop(outliers_z_score(income_log).index)

*В дальнейшем применим к этому признаку `StandardScaler`.*

In [ ]:
std_cols.append('income')
log_cols.append('income')

### `foreign_passport`

Флаг наличия загранпаспорта у клиента. Бинарный признак.

Визуализируем его в разрезе дефолт/не-дефолт.

In [ ]:
display(train_df['foreign_passport'].value_counts(normalize=True) * 100)
build_histplot(train_df['foreign_passport'], train_df['default']);

In [ ]:
default_by_foreign_passport = train_df.groupby('foreign_passport', as_index=False)['default'].mean()

display(default_by_foreign_passport)
build_barplot(default_by_foreign_passport['foreign_passport'], default_by_foreign_passport['default']);

Из построенных графиков видим, что большая часть клиентов (85%) не имеют загранпаспорта. При этом процент дефолтов у клиентов без загранпаспорта почти в 2 раза выше, чем у клиентов с загранпаспортом (13% против 7%).

*В будущем перекодируем этот признак в цифровой формат.*

In [ ]:
bin_cols.append('foreign_passport')

### `sna`

Связь заемщика с клиентами банка. Категориальный признак.

Признак уже закодирован в цифровом формате. К сожалению, расшифровки для каждой категории нет. Визуализируем их в разрезе дефолт/не-дефолт.

In [ ]:
display(train_df['sna'].value_counts(normalize=True) * 100)
build_histplot(train_df['sna'], train_df['default'], bins=train_df['sna'].nunique());

In [ ]:
default_by_sna = train_df.groupby('sna', as_index=False)['default'].mean()

display(default_by_sna)
build_barplot(default_by_sna['sna'], default_by_sna['default']);

Из построенных графиков видим, что соотношение категорий 1/2/3/4 примерно 65/15/5/15. Прослеживается некоторая зависимость платёжеспособности клиента от категории. Самая многочисленная категория (1) имеет самый низкий процент дефолтов (10%), а категория 4 имеет дефолт в 2 раза чаще (20%). Две другие категории имеют около 15% дефолтов.

*Закодируем этот признак в будущем с помощью `OneHotEncoding`, т.к. нет информации о том, является ли 1 категория качественно хуже 2, 3 или 4.*

In [ ]:
one_hot_cols.append('sna')

### `first_time`

Давность наличия информации о заемщике. Категориальный признак.

К сожалению, информация о значении каждой категории не предоставлена. Визуализируем его в разрезе дефолт/не-дефолт.

In [ ]:
display(train_df['first_time'].value_counts(normalize=True) * 100)
build_histplot(train_df['first_time'], train_df['default'], bins=train_df['first_time'].nunique());

In [ ]:
default_by_first_time = train_df.groupby('first_time', as_index=False)['default'].mean()

display(default_by_first_time)
build_barplot(default_by_first_time['first_time'], default_by_first_time['default']);

Из графиков и таблиц видим, что соотношение категорий клиентов 1/2/3/4 примерно 17/16/42/25. При этом прослеживается связь между платёжеспособностью клиента и этим признаком. Для категории 1 доля дефолтов достигает 18%, в то время как для категории 4 этот показатель составляет 8%. Для категорий 2 и 3 доля дефолтов соответственно 15% и 12%.

*В будущем дополнительно закодируем этот признак с помощью `OneHotEncoding`, т.к. неизвестно, является ли признак с значением 1 качественно хуже признака с значением 2+*

In [ ]:
one_hot_cols.append('first_time')

### `score_bki`

Скоринговый балл по данным из БКИ. Числовой признак.

Исследуем его, визуализируем в разрезе дефолт/не-дефолт, проверим связь показателя с `bki_request_cnt`.

In [ ]:
build_distplots(train_df['score_bki'], train_df['default']);

In [ ]:
build_boxplot(train_df['score_bki'], train_df['bki_request_cnt']);

Проверим распределение на нормальность.

In [ ]:
check_normal_distribution(train_df['score_bki']);

Следуя выводам из теста и смотря на график распределения, будем считать, что имеем дело с данными, близким к нормальному распределению. Также на гистограмме и боксплотах видим, что у дефолтных клиентов в среднем рейтинг ближе к 0, чем у не-дефолтных. Помимо этого видим связь с признаком `bki_request_cnt` - чем больше количество заявок, тем ниже средний рейтинг клиентов.

Т.к. распределение данных близко к нормальному, произведём очистку от выбросов методом z-отклонений.

In [ ]:
train_df = train_df.drop(outliers_z_score(train_df['score_bki']).index)

*В будущем осуществим преобразование признака с помощью `StandardScaler`.*

In [ ]:
std_cols.append('score_bki')

### `region_rating`

Рейтинг региона. Категориальный признак.

Визуализируем его в разрезе дефолт/не-дефолт, оценим соотношение. Вероятно может присутствовать связь с другими показателями (`income`, `good_work`, `age`, `home_address`, `work_address`, `score_bki`)

In [ ]:
display(train_df['region_rating'].value_counts(normalize=True) * 100)
build_histplot(train_df['region_rating'], train_df['default'], bins=train_df['region_rating'].nunique());

In [ ]:
default_by_region_rating = train_df.groupby('region_rating', as_index=False)['default'].mean()

display(default_by_region_rating)
build_barplot(default_by_region_rating['region_rating'], default_by_region_rating['default']);

In [ ]:
build_boxplot(train_df['age'], train_df['region_rating']);

In [ ]:
build_boxplot(train_df['income'], train_df['region_rating']);

In [ ]:
good_work_by_region_rating = train_df.groupby('region_rating', as_index=False)['good_work'].mean()

display(good_work_by_region_rating)
build_barplot(good_work_by_region_rating['region_rating'], good_work_by_region_rating['good_work']);

Мы могли бы исследовать связь `region_rating` с `home_address` и `work_address`, однако из-за отсутствия информации по данным признакам, это имеет мало смысла.

In [ ]:
# train_df['region_home_address_fraction'] = train_df.groupby(['home_address', 'region_rating'])['home_address'].transform('count') / train_df.groupby('region_rating')['region_rating'].transform('count') * 100

# display(np.round(train_df.groupby(['region_rating', 'home_address'], as_index=False)['region_home_address_fraction'].mean(), 2))
# build_histplot(train_df['region_rating'], train_df['home_address'], bins=train_df['region_rating'].nunique(), palette='flare');

# train_df = train_df.drop('region_home_address_fraction', axis=1)

In [ ]:
# train_df['region_work_address_fraction'] = train_df.groupby(['work_address', 'region_rating'])['work_address'].transform('count') / train_df.groupby('region_rating')['region_rating'].transform('count') * 100

# display(np.round(train_df.groupby(['region_rating', 'work_address'], as_index=False)['region_work_address_fraction'].mean(), 2))
# build_histplot(train_df['region_rating'], train_df['work_address'], bins=train_df['region_rating'].nunique(), palette='flare');

# train_df = train_df.drop('region_work_address_fraction', axis=1)

In [ ]:
build_boxplot(train_df['score_bki'], train_df['region_rating']);

По построенным графикам можем видеть, что наименьшее количество клиентов (<1%) находится в регионах с рейтингом 20 и 30, большая часть (около 75%) сосредоточена в регионах с рейтингом 40-60, оставшаяся часть находится в регионах с рейтингом 70-80.

Прослеживается зависимость, что чем выше рейтинг региона, тем меньше в нём доля дефолтных клиентов - почти 20% в регионе с рейтингом 20 и постепенное снижение до 7% в регионе с рейтингом 80.

С повышением рейтинга региона повышаются и средние доходы клиентов, а также понижается средний возраст клиента.

По флагу "хорошей" работы можно сделать вывод, что в регионах с низким и высоким рейтингом доля клиентов с "хорошей" работой выше, чем в регионах со средним рейтингом.

Средний рейтинг БКИ от региона к региону не сильно меняется, поэтому сложно проследить какую-то зависимость, кроме того, что в регионе с рейтингом 20 средний рейтинг БКИ дальше от нуля по сравнению с остальными. Но и разброс значений там меньше. Разброс в регионе с рейтингом 30 аналогичен, а средние значения ближе к нулю.

*В будущем произведём кодировку с помощью `OneHotEncoder`.*

*Потенциальный вариант улучшения качества модели - сокращение количества категорий до 3 (20-30: низкий, 40-60: средний, 70-80: высокий)*

In [ ]:
one_hot_cols.append('region_rating')

### `app_date`

Дата подачи заявки.

Приведём признак к формату даты и исследуем его.

In [ ]:
train_df['app_date'] = pd.to_datetime(train_df['app_date'])
train_df[['app_date']].describe()

Данные приведены за 120 дней с 1 января по 30 апреля. Проверим их распределение в разрезе дефолт/не-дефолт.

In [ ]:
build_histplot(train_df['app_date'], train_df['default'], bins=train_df['app_date'].nunique());

In [ ]:
default_by_app_date = train_df.groupby('app_date', as_index=False)['default'].mean()

display(default_by_app_date)
build_barplot(default_by_app_date['app_date'].dt.date, default_by_app_date['default'], figsize=(18, 5)).tick_params('x', labelrotation=60);


Из построенных графиков видим, что ближе зимой общее количество заявок немного меньше, чем весной. К концу марта количество заявок немного увеличивается. Можно было предположить, что в предпраздничные дни количество заявок может увеличиваться, но из графиков никакой зависимости от дат не прослеживается (за исключением естественного проседания каждые в выходные).
В начале января есть небольшой скачок дефолтных клиентов, но дальше график идёт примерно одинаково и доля дефолтных клиентов варьируется в районе 10-15% вне зависимости от даты подачи. Также видим, что доля дефолтных клиентов с датой подачи заявки в апреле ниже, чем в январе. Попробуем разделить клиентов по месяцам подачи заявки.

In [ ]:
train_df['app_month'] = train_df['app_date'].dt.month

default_by_app_month = train_df.groupby('app_month', as_index=False)['default'].mean()
display(default_by_app_month)
build_barplot(default_by_app_month['app_month'], default_by_app_month['default']);

Из построенного графика видим, что доля дефолтных клиентов с датой подачи заявки в апреле ниже. Однако, вряд ли этот факт можно считать чем-то значимым, т.к. у нас отсутствуют данные за остальные месяцы.

*Делаем вывод, что данный признак бесполезен для нашей модели, поэтому удалим его, а также в будущем удалим признак даты подачи заявки*

In [ ]:
train_df = train_df.drop('app_month', axis=1)
cols_to_delete.append('app_date')

Реализуем все выводы выше о преобразованиях признаков в функции-трансформере и применим её на тренировочном и тестовом наборах.

In [ ]:
display(
    train_df.head(),
    train_df.describe(),
    train_df.describe(include=['object']),
    train_df.info(),
    std_cols,
    robust_cols,
    one_hot_cols,
    ordinal_cols,
    bin_cols,
    cols_to_delete,
)

In [ ]:
def transform_features(data, transformers, fit=False):
    result = data.copy()
    
    car_filter = data['car'] == 'Y'
    foreign_car_filter = data['car_type'] == 'Y'
    result['has_domestic_car'] = (car_filter & ~foreign_car_filter).astype('uint8')
    result['has_foreign_car'] = (car_filter & foreign_car_filter).astype('uint8')
    
    for col in log_cols:
        result[col] = np.log(result[col])
    
    for col in na_fill_values:
        result[col] = result[col].fillna(na_fill_values[col])
    
    for transformer, cols in transformers:
        if fit: transformer.fit(result[cols])

        result[
            transformer.get_feature_names_out(cols) if hasattr(transformer, 'get_feature_names_out') else cols
        ] = transformer.transform(result[cols])

    return result

Подготовим энкодеры/скейлеры, преобразуем данные и разделим датасет и целевую переменную.

In [ ]:
ordinal_encoder = OrdinalEncoder(
    categories=[
        ['SCH', 'UGR', 'GRD', 'PGR', 'ACD'],
        [20, 30, 40, 50, 60, 70, 80]
    ],
    dtype='uint8'
)
one_hot_encoder = OneHotEncoder(sparse=False, dtype='uint8')
bin_encoder = OrdinalEncoder(dtype='uint8')
std_scaler = StandardScaler()
robust_scaler = RobustScaler()

transformers = [
    (one_hot_encoder, one_hot_cols),
    (bin_encoder, bin_cols),
    (std_scaler, std_cols),
    (robust_scaler, robust_cols),
]

X = transform_features(train_df, transformers, fit=True).drop(cols_to_delete + one_hot_cols + ['default'], axis=1)
y = train_df['default']

Построим матрицу корреляций для числовых и категориальных признаков, проверим данные на мультиколлинеарность. Будем использовать корреляции Пирсона и Спирмена для соответствующих признаков.

In [ ]:
def corr_heatmaps(df_list):
    fig = plt.figure(figsize=(18, 36))
    axes = fig.subplots(2, 1)
    
    for i, (df, method) in enumerate(df_list):
        seaborn.heatmap(df.corr(method=method), annot=True, ax=axes[i])
    
    return axes

cat_cols = ordinal_cols + bin_cols + list(one_hot_encoder.get_feature_names(one_hot_cols)) + ['has_domestic_car', 'has_foreign_car']
num_cols = robust_cols + std_cols

corr_heatmaps([(X[cat_cols], 'pearson'), (X[num_cols], 'spearman')]);

Среди числовых признаков мультиколлинеарности нет. Среди категориальных наблюдается высокая корреляция категорий `work_address` и `home_address`. В этом нет ничего удивительного - одному типу рабочего адреса с большей вероятностью вполне может соответствовать какой-то тип домашнего адреса. Оставим это пока как есть.

Проведём статистические тесты на связь данных с целевым признаком. Будем использовать дисперсионный анализ для числовых признаков и хи-квадрат для категориальных.

In [ ]:
def build_imp_barplot(data, target, test=chi2):
    imp = pd.DataFrame({ 'feature': data.columns, 'importance': test(data, target)[0] }).sort_values('importance', ascending=False)
    imp_max = imp['importance'].max()

    return build_barplot(
        imp['feature'],
        imp['importance'],
        scale=(0, imp_max),
        step=imp_max // 10,
    )

build_imp_barplot(X[cat_cols], y).tick_params('x', labelrotation=90);

In [ ]:
build_imp_barplot(X[num_cols], y, test=f_classif);

По статистическим тестам можем сделать вывод, что в состав наиболее влиятельных категориальных признаков входят `education`, `region_rating`, `foreign_passport`, `has_foreign_car`, `sna_4,1`, `first_time_1,4`, `home_address_1,2`. Признаки `good_work`, `sex`, `has_domestic_car` и др. являются наименее значимыми.

К самым влиятельным числовым признакам относятся `score_bki` и `decline_app_cnt`.

Данные готовы к обучению.

In [ ]:
display(y.value_counts(normalize=True) * 100)
build_histplot(y, bins=y.nunique());

Как видим из графика, мы имеем дело с несбалансированной выборкой. Будем делать стратифицированную выборку, обучение произведём с холд-аут выборкой. Модель обучения - логистическая регрессия, в параметрах укажем о необходимости балансировки классов.

In [ ]:
log_reg_model = LogisticRegression(max_iter=1000, class_weight='balanced')

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, test_size=0.2)

log_reg_model.fit(X_train, y_train)
y_train_pred = log_reg_model.predict(X_train)
y_valid_pred = log_reg_model.predict(X_valid)

In [ ]:
def confusion_matrix_plot(y_true, y_pred, title='Матрица ошибок', ax=None):
    plot = seaborn.heatmap(data=confusion_matrix(y_true, y_pred), annot=True, cmap='Blues', ax=ax)
    plot.set_title(title)
    plot.set_xlabel('Истинное значение')
    plot.set_ylabel('Предсказание')

    return plot

print(classification_report(y_valid, y_valid_pred))
confusion_matrix_plot(y_valid, y_valid_pred);

Качество модели неоднозначное. На матрице ошибок видим немалое количество ложноотрицательных результатов (лишь пятая часть дефолтов была верно предсказана), а также видим, что около 60% предсказанных дефолтов на самом деле являются таковыми, т.е. имеем 40% ложноположительных предсказаний. Попробуем подобрать параметры для определения клиента в тот или иной класс (т.е. подкрутим порог оценки вероятности).

In [ ]:
def f1_by_proba_plot(model, X, y_true):
    y_proba_pred = pd.Series(model.predict_proba(X)[:, 1])

    f1_scores = []

    thresholds = np.arange(0.1, 1, 0.05)

    for threshold in thresholds:
        f1_scores.append(f1_score(y_true, y_proba_pred.apply(lambda p: 1 if p > threshold else 0)))
        
    ax = get_axes(figsize=(10, 5))
    
    ax.plot(thresholds, f1_scores, label='F1-score')

    ax.set_title('F1-score в зависимости от порога вероятности')
    ax.set_xlabel('Порог вероятности')
    ax.set_ylabel('F1-score')
    ax.set_xticks(thresholds)

    ax.legend()
    ax.grid()

    return ax
    
f1_by_proba_plot(log_reg_model, X_valid, y_valid);

Как видим на графике, наиболее высокое значение метрики F1-score находится в районе оценки вероятности 0.60. Построим PR-кривую и с её помощью определим оптимальный порог.

In [ ]:
def get_opt_f1_score_proba(model, X, y_true):
    precision, recall, thresholds = precision_recall_curve(y_true, model.predict_proba(X)[:, 1])
    max_score_idx = np.argmax((2 * precision * recall) / (precision + recall))

    return thresholds[max_score_idx]
    
opt_proba = get_opt_f1_score_proba(log_reg_model, X_valid, y_valid)

def build_pr_curve_plot(model, X, y_true, opt_f1_score_thresh=None):
    precision, recall, thresholds = precision_recall_curve(y_true, model.predict_proba(X)[:, 1])

    axes = get_axes()
    axes.plot(precision, recall, label='PR-кривая')
    axes.set_title('PR-кривая')
    axes.set_xlabel('Recall')
    axes.set_ylabel('Precision')
    axes.grid()
    axes.legend()
    
    if opt_f1_score_thresh:
        idx = list(thresholds).index(opt_f1_score_thresh)
        axes.scatter(precision[idx], recall[idx], marker='o', color='black', label='Лучший F1-score')
        
    return axes

build_pr_curve_plot(log_reg_model, X_valid, y_valid, opt_proba);

Мы определили оптимальный порог оценки вероятности для модели. Выведем метрики с учётом этого изменения.

In [ ]:
y_valid_pred = pd.Series(log_reg_model.predict_proba(X_valid)[:, 1]).apply(lambda p: 1 if p >= opt_proba else 0)

print(classification_report(y_valid, y_valid_pred))
confusion_matrix_plot(y_valid, y_valid_pred);

Сдвинув порог оценки вероятности была незначительно улучшена метрика F1. Немного понизился recall (доля ложноотрицательных предсказаний снизилась), и немного увеличился precision (повысилась доля ложноположительных предсказаний). Также примерно на 10% повысилась accuracy.

Используем эту модель для предсказания тестовых данных.

In [ ]:
X_test = transform_features(test_df, transformers).drop(cols_to_delete + one_hot_cols, axis=1)
submission['default'] = pd.Series(log_reg_model.predict_proba(X_test)[:, 1]).apply(lambda p: 1 if p >= opt_proba else 0)

In [ ]:
submission.describe()

In [ ]:
submission.to_csv('submission.csv', index=False)

Итого:

Пропуски в образовании заполнены модой (SCH).

Количество подач заявок отфильтровано от выбросов методом Тьюки и отскейлено `RobustScaler`ом.

Количество отказов отфильтровано руками и отскейлен `RobustScaler`ом.

Доход и возраст очищены от выбросов методом z-отклонений и стандартизированы (доход прологарифмирован).

Все категориальные признаки закодированы с помощью `OneHotEncoding`.

Осуществили балансировку классов.

Порог оценки вероятности подобран с помощью PR-кривой с наибольшим значением f1.

*(Был опробован отбор наиболее значимых признаков, кодировка образования и рейтинга региона в порядке возрастания, уменьшение количества категорий в рейтинге региона, оверсемплинг SMOTE, конвертация количества отказов и подач заявок в категориальные признаки, подбор вероятности для несбалансированной выборки)*